# Unifying the datasets and create a cache for w2v vectores




In this work, we deal with 4 different datasets  PAN-CLEF2018 train (pre-competition),  PAN-CLEF2018 eval (the actual competition), Lyrics dataset and the socialaa.  Each of the these datasets  was in a slight different configuration, demanding  various pre-processing steps.  This code is meant to unify and also creating a filtered word2vec pre-trained.

# Filtering the word2vec

Pre-trained models are usually huge files what could led to a slow training process.

In [ ]:
import os;
import re;

import pandas as pd;

import zipfile;
import json;

from time import time

In [ ]:
!pip install gensim=='3.8.0'

In [ ]:
import gensim
from  gensim.models import KeyedVectors

# Unifying all dataset

Converting PAN-CLEF dataset into a row style dataframe.

In [ ]:
def readCollectionsOfProblemsZip(zip_file, project_name):
    infocollection = 'collection-info.json';
    
    problems = json.loads(zip_file.read(infocollection).decode('utf-8'));
    candidates = []
    for problem in problems:
        problem_info = '/'.join([problem['problem-name'], 'problem-info.json'])
        problem_info_obj = json.loads(zip_file.read(problem_info).decode('utf-8'));
        for c in problem_info_obj['candidate-authors']:
            candidate =  {'problem':problem['problem-name'], 'language':problem['language'],'label': c['author-name']};
            c_path = '/'.join([problem['problem-name'], c['author-name']]);
            
            for f in zip_file.filelist :
                if f.is_dir() or  not f.filename.endswith('txt'):
                    continue;
                if c_path in f.filename:
                    #print(f.filename)
                    
                    candidates.append({
                        **candidate,
                        **{
                            'text':zip_file.read(f).decode(problem['encoding']),
                            'set':'known',
                            'filename':f.filename.split('/')[-1]
                        }
                    });
                    
        # reading the test set for each problem
        u_path = '/'.join([problem['problem-name'], 'ground-truth.json']);
        unknow_set_obj = json.loads(zip_file.read(u_path))['ground_truth'];
        
        for f in unknow_set_obj:
            u_path = '/'.join([problem['problem-name'], 'unknown',f['unknown-text']]);
            #print(u_path, f)
            candidate =  {'problem':problem['problem-name'], 'language':problem['language'],'label': f['true-author']};
            
            candidates.append({
                        **candidate,
                        **{
                            'text':zip_file.read(u_path).decode(problem['encoding']),
                            'set':'unknown',
                            'filename':f['unknown-text']
                        }
                    });
        
        
   
        #for f in zf.filelist :
        
        
    return candidates;
    
def readDataSet(project_name):
    project_path = f'../../data/{project_name}.zip';
    
    with zipfile.ZipFile(project_path) as zf:
        df = pd.DataFrame(readCollectionsOfProblemsZip(zf, project_name));
        
    return df;

In [ ]:
PAN18_train = readDataSet('pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02');

In [ ]:
PAN18_test = readDataSet('pan18-cross-domain-authorship-attribution-test-dataset2-2018-04-20');

In [ ]:
lyrics = readDataSet('Lyrics_PT_EN_2019');

In [ ]:
lyrics.head(10)

,filename,label,language,problem,set,text
0,known00008.txt,candidate00001,pt,problem00001,known,Chega simples como um temporal\nParecia que ia...
1,known00020.txt,candidate00001,pt,problem00001,known,Houve um tempo em que os homens\nEm suas tribo...
2,known00021.txt,candidate00001,pt,problem00001,known,Te vejo errando e isso não é pecado\nExceto qu...
3,known00009.txt,candidate00001,pt,problem00001,known,Eu queria era dizer diferente\nAquilo que todo...
4,known00023.txt,candidate00001,pt,problem00001,known,O êxito tem vários pais\nÓrfão é o seu revés\n...
5,known00022.txt,candidate00001,pt,problem00001,known,"Da a pata, senta, deita\nNão respire, apague a..."
6,known00019.txt,candidate00001,pt,problem00001,known,- Memórias \npitty \nEu fui matando os meus...
7,known00018.txt,candidate00001,pt,problem00001,known,Só nos últimos cinco meses\nEu já morri umas q...
8,known00001.txt,candidate00001,pt,problem00001,known,Tantas decepções eu já vivi\nAquela foi de lon...
9,known00015.txt,candidate00001,pt,problem00001,known,Quem não tem teto de vidro\nQue atire a primei...


In [ ]:
def readSocial(path):
    with zipfile.ZipFile(path) as zf:
        temp = json.loads(zf.read('social_media_aa.json'))
        social = [];
        for p in temp:
            problem = {'problem':p['problem-name'],'language':p['language']}
            for c in p['candidates']:
                social.append({
                    **problem,
                    **{'text':c[0],'label':c[1],'filename':c[2],'set':'known',}
                })
                
            for c in p['unknown']:
                social.append({
                    **problem,
                    **{'text':c[0],'label':c[1],'filename':c[2],'set':'unknown',}
                })
                
    return pd.DataFrame(social)
            
social_AA = readSocial('../../data/social_media_aa.json.zip');
social_AA.head()

,filename,label,language,problem,set,text
0,f0,candidate00001,en,problem00001,known,I don't have time to be depressed
1,f1,candidate00001,en,problem00001,known,REF this lady had to break the chickens neck a...
2,f2,candidate00001,en,problem00001,known,"“REF Come get ya man...REF URL""””""” 😂😂😂😂😂😂"
3,f3,candidate00001,en,problem00001,known,“REF Mini me is asleep.. Now what..lol” TURN U...
4,f4,candidate00001,en,problem00001,known,A nap is in order


In [ ]:
def addSetName(df, name):
    df = df.copy();
    df['dataset'] = name;
    return df
allDS = pd.concat([
    addSetName(PAN18_train,'pan18_train'),
    addSetName(PAN18_test,'pan18_eval'),
    addSetName(lyrics,'lyrics'),
    addSetName(social_AA,'socialaa')
])

In [ ]:
allDS

,filename,label,language,problem,set,text,dataset
0,known00001.txt,candidate00001,en,problem00001,known,"graceful ones.\n\n""One more,"" Marvelous said, ...",pan18_train
1,known00002.txt,candidate00001,en,problem00001,known,"before. If he can, he’ll remember a classmate ...",pan18_train
2,known00003.txt,candidate00001,en,problem00001,known,she thought - he was in Team Baron only becaus...,pan18_train
3,known00007.txt,candidate00001,en,problem00001,known,"As far as she remembers, she's always hated pr...",pan18_train
4,known00006.txt,candidate00001,en,problem00001,known,"“Wait for me, please!”\n\nShe glanced towards ...",pan18_train
5,known00004.txt,candidate00001,en,problem00001,known,Zawame City’s no longer ‘home’ as it once was....,pan18_train
6,known00005.txt,candidate00001,en,problem00001,known,1. He showered her with smiles the first time ...,pan18_train
7,known00001.txt,candidate00002,en,problem00001,known,"love?"" The words should have been scalding, bu...",pan18_train
8,known00002.txt,candidate00002,en,problem00001,known,"'Asha Belarl'an', just as you are?""\n\n""Kalli,...",pan18_train
9,known00003.txt,candidate00002,en,problem00001,known,"EinsOne night, in a sleazy bar on Coridan, an ...",pan18_train


In [ ]:
allDS.to_json('../../data/AllDS.json.zip', orient='records', compression='gzip')

In [ ]:
allDS.groupby('language').count()

,filename,label,problem,set,text,dataset
language,,,,,,
en,24657,24657,24657,24657,24657,24657
fr,833,833,833,833,833,833
it,888,888,888,888,888,888
pl,943,943,943,943,943,943
pt,2608,2608,2608,2608,2608,2608
sp,1037,1037,1037,1037,1037,1037


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Caching embeddings

Embedding files are huge and heavy files. Reading them takes a lot of time.  This code is meant to create a vocabulary with all possible occurrences in the presente work and filter the original W2V into a small sample  in order to speed up  the train/test process.

|ID|Download link|Vector size|Window|Corpus|Vocabulary size|Algorithm|Lemmatization|
|--- |--- |--- |--- |--- |--- |--- |--- |
|43|http://vectors.nlpl.eu/repository/20/43.zip|100|10|French CoNLL17 corpus|2567698|Word2Vec Continuous Skipgram|False|
|52|http://vectors.nlpl.eu/repository/20/52.zip|100|10|Italian CoNLL17 corpus|2469122|Word2Vec Continuous Skipgram|False|
|62|http://vectors.nlpl.eu/repository/20/62.zip|100|10|Polish CoNLL17 corpus|4420598|Word2Vec Continuous Skipgram|False|
|63|http://vectors.nlpl.eu/repository/20/63.zip|100|10|Portuguese CoNLL17 corpus|2536452|Word2Vec Continuous Skipgram|False|
|68|http://vectors.nlpl.eu/repository/20/68.zip|100|10|Spanish CoNLL17 corpus|2656057|Word2Vec Continuous Skipgram|False|

In [ ]:
def buildVocabulary(texts):
    counter = CountVectorizer(analyzer='word', lowercase=False);
    counter.fit(texts);
    return counter.vocabulary_;

In [ ]:
def loadEnModel():
    return KeyedVectors.load_word2vec_format(
        os.path.join('GoogleNews-vectors-negative300.bin.gz')
        ,binary=True);

In [ ]:
def loadModel(filename):
    with zipfile.ZipFile(filename, "r") as archive:
        stream =  archive.open("model.txt");
        model = KeyedVectors.load_word2vec_format(stream, binary=False, unicode_errors='replace')
    return model

In [ ]:
def createEmbeddingCache(model, vocab, fname):
    
    print("Filtering model",end=' ');t0 = time();
    vocabFilter = {};
    vocabFilter = {w:model[w] for w in vocab if w in model};
    print("Done in %0.3fs" % (time() - t0))
    
    #embeddingSize= len(model[vocabFilter[0]])
    embeddingSize = model.vector_size;
    
    #trying to free the memory
    del model;
    import gc;
    gc.collect();
    
    print({'embeddingSize':embeddingSize, 'vocabSize':len(vocab), 'vocabFound':len(vocabFilter),'per_found':len(vocabFilter)/len(vocab)})
    
    
    print("Writing model",end=' ');t0 = time();
    with gzip.open(os.path.join('../embedding_cache',fname), 'w') as f:
        f.write(("%s %s\n"%(len(vocabFilter),embeddingSize)).encode('utf-8'))
        for w in sorted(list(vocabFilter.keys())):
            a = " ".join([str(f) for f in vocabFilter[w]]);
            line = "%s %s\n" % (w, a)
            f.write(line.encode('utf-8'))
    print("Done in %0.3fs" % (time() - t0));

## caching english

In [ ]:
print('Loading google w2v english - start');t0 = time();
model = loadEnModel();
print("Done in %0.3fs" % (time() - t0))

vocab = buildVocabulary(allDS.query('language == "en"')['text']);
            
createEmbeddingCache(model, vocab, f'w2v_en.txt.gz')

Loading google w2v english - start
Done in 155.712s
Filtering model Done in 3.823s
{'embeddingSize': 300, 'vocabSize': 60474, 'vocabFound': 48560, 'per_found': 0.8029897145880874}
Writing model Done in 36.743s


# Caching the other languages

In [ ]:
otherModels = {
    'fr':'43',
    'it':'52',
    'pl':'208',
    'pt':'63',
    'pl':'62',
    'sp':'68',
}

In [ ]:
for lang, filenumber in otherModels.items():
    print(f'\n\nLoading  w2v {lang} - start');t0 = time();
    model = loadModel(f'{filenumber}.zip');
    print("Done in %0.3fs" % (time() - t0))

    vocab = buildVocabulary(allDS.query(f'language == "{lang}"')['text']);

    createEmbeddingCache(model, vocab, f'w2v_{lang}.txt.gz')



Loading  w2v fr - start
Done in 289.497s
Filtering model Done in 0.093s
{'embeddingSize': 100, 'vocabSize': 24698, 'vocabFound': 20232, 'per_found': 0.8191756417523686}
Writing model Done in 7.101s


Loading  w2v it - start
Done in 272.104s
Filtering model Done in 0.093s
{'embeddingSize': 100, 'vocabSize': 30291, 'vocabFound': 22817, 'per_found': 0.7532600442375623}
Writing model Done in 7.435s


Loading  w2v pl - start
Done in 510.913s
Filtering model Done in 0.262s
{'embeddingSize': 100, 'vocabSize': 54183, 'vocabFound': 41885, 'per_found': 0.7730284406548179}
Writing model Done in 13.727s


Loading  w2v pt - start
Done in 296.719s
Filtering model Done in 0.103s
{'embeddingSize': 100, 'vocabSize': 20068, 'vocabFound': 14306, 'per_found': 0.7128762208491131}
Writing model Done in 4.779s


Loading  w2v sp - start
Done in 317.376s
Filtering model Done in 0.096s
{'embeddingSize': 100, 'vocabSize': 30807, 'vocabFound': 23222, 'per_found': 0.7537897231148765}
Writing model Done in 7.928s